<a href="https://colab.research.google.com/github/NavaneethaCSR/Covid-Detection-using-CT-scan-Images/blob/main/Covid_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from keras.preprocessing import image
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Input,Conv2D, MaxPooling2D,Activation,AveragePooling2D
from tensorflow.keras import layers, models,regularizers
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
m, n = 224,224
train_data='/content/drive/MyDrive/Colab Notebooks/Datasets/covid_dec/train'
test_data='/content/drive/MyDrive/Colab Notebooks/Datasets/covid_dec/test'
validation_data='/content/drive/MyDrive/Colab Notebooks/Datasets/covid_dec/validation'

# **PRE-PROCESSING**

In [ ]:
import cv2
import os
import numpy as np

def compress_image(image, quality=95):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, compressed_image = cv2.imencode('.jpg', image, encode_param)
    decoded_image = cv2.imdecode(compressed_image, 1)
    return decoded_image

def ela(image, quality=95):
    compressed_image = compress_image(image, quality)
    grayscale_image = cv2.cvtColor(compressed_image, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(grayscale_image)
    ela_image = cv2.absdiff(grayscale_image, equalized_image)
    return ela_image

def preprocess_image(image_path, target_size=(224, 224), quality=95):
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            raise ValueError("Failed to load image: {}".format(image_path))

        resized_image = cv2.resize(original_image, target_size)

        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(resized_image, (5, 5), 0)

        #convert to grayscale
        gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)


        # Apply Sobel edge detection

        sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=5)
        sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=5)
        edge_image = cv2.magnitude(sobel_x, sobel_y)


        # Apply histogram equalization for contrast enhancement
        equalized_image = cv2.equalizeHist(gray_image)

        # Apply ELA
        ela_result = ela(resized_image, quality)

        # Combine preprocessing results
        preprocessed_image = np.concatenate((blurred_image, np.expand_dims(edge_image, axis=-1), np.expand_dims(ela_result, axis=-1)), axis=-1)

        # Normalize pixel values to the range [0, 1]
        preprocessed_image = preprocessed_image / 255.0

        return preprocessed_image
    except Exception as e:
        print("Error processing image {}: {}".format(image_path, e))
        return None

def preprocess_images_from_folders(data_dir, target_size=(224, 224), quality=95):
    preprocessed_images = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(label_dir, filename)
                    preprocessed_image = preprocess_image(image_path, target_size, quality)
                    if preprocessed_image is not None:
                        preprocessed_images.append(preprocessed_image)
                        labels.append(label)

    return preprocessed_images, labels

In [ ]:
# Preprocess training images
train_images, train_labels = preprocess_images_from_folders(train_data)

# Preprocess validation images
validation_images, validation_labels = preprocess_images_from_folders(validation_data)

# Preprocess test images
test_images, test_labels = preprocess_images_from_folders(test_data)

In [ ]:
import numpy as np

# Convert preprocessed images and labels to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels to numerical format
train_labels_encoded = label_encoder.fit_transform(train_labels)
validation_labels_encoded = label_encoder.transform(validation_labels)

test_labels_encoded = label_encoder.transform(test_labels)

In [ ]:
train_labels_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
from tensorflow.keras.regularizers import l2
model = Sequential()

# Convolutional layers (reduced filters and added dropout)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 5)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))  # Dropout after first convolutional block
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))  # Dropout after second convolutional block

# Flatten and dense layers (reduced neurons and added L2 regularization)
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile model
import tensorflow as tf
custom_optimizer = Adam(learning_rate=0.001, beta_1=0.9)
model.compile(optimizer=custom_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
#If the training loss continues to decrease while the validation loss increases, it's a sign of overfitting. However, if both losses decrease simultaneously or plateau together, it indicates that the model is not overfitting.

In [ ]:
#If the training accuracy continues to improve while the validation accuracy stagnates or decreases, it suggests overfitting. Balanced improvements or plateaus in both accuracies indicate no overfitting.

#**TRAINING**

In [ ]:
model.fit(train_images, train_labels_encoded, epochs=20, batch_size=16,validation_data=(validation_images, validation_labels_encoded),callbacks=[early_stopping])

Epoch 1/20
7/7 [==============================] - 12s 2s/step - loss: 1.2394 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.8551 - val_accuracy: 0.3500 - val_precision: 0.2857 - val_recall: 0.2000
Epoch 2/20
7/7 [==============================] - 11s 2s/step - loss: 1.1793 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0159 - val_accuracy: 0.3000 - val_precision: 0.2500 - val_recall: 0.2000
Epoch 3/20
7/7 [==============================] - 10s 1s/step - loss: 1.1146 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0959 - val_accuracy: 0.3000 - val_precision: 0.2500 - val_recall: 0.2000
Epoch 4/20
7/7 [==============================] - 12s 2s/step - loss: 1.0536 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.1340 - val_accuracy: 0.3500 - val_precision: 0.2857 - val_recall: 0.2000


# **TESTING**

In [ ]:
evaluation_results = model.evaluate(test_images,test_labels_encoded)

1/1 [==============================] - 0s 423ms/step - loss: 1.7854 - accuracy: 0.7000 - precision: 0.7500 - recall: 0.6000


In [ ]:
import math
predictions = model.predict(test_images)
n=np.mean(predictions)
# Get the actual labels
actual_labels = test_labels

# Print the predicted and actual labels
for i in range(len(predictions)):
    predicted_label = 'non covid' if predictions[i]>= n else ' covid'

    print("Predicted Label:", predicted_label, "Actual Label:", actual_labels[i])

1/1 [==============================] - 0s 421ms/step
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label: non covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label:  covid Actual Label: covid
Predicted Label: non covid Actual Label: covid
Predicted Label:  covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label: non covid Actual Label: non_covid
Predicted Label:  covid Actual Label: non_cov

In [ ]:
for i in range(len(predictions)):
  print(predictions[i])

[0.09088783]
[0.02099293]
[0.00068105]
[0.05360046]
[0.02397937]
[0.85481423]
[0.18815817]
[0.02121178]
[0.06538866]
[0.641206]
[0.06800812]
[0.87539524]
[0.9913794]
[0.9596228]
[0.8158789]
[0.4798437]
[0.98842895]
[0.5333134]
[0.49616247]
[0.03882167]


In [ ]:
pred_targets = np.where(predictions >= n, 1, 0)


In [ ]:
true_labels = label_encoder.fit_transform(actual_labels)

In [ ]:
confusion_test = confusion_matrix(true_labels, pred_targets)

In [ ]:
confusion_test

array([[8, 2],
       [2, 8]])